<a href="https://colab.research.google.com/github/ishammansoor/AI-and-Machine-Learning/blob/main/FineTuning_gpt2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ✅ STEP 1: Install necessary packages
!pip install transformers datasets accelerate bitsandbytes -q
!pip install -U datasets fsspec -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [1]:
# ✅ STEP 2: Load GSM8K with Chain-of-Thought
from datasets import load_dataset
dataset = load_dataset("gsm8k", "main")

# Example formatting: Q: ... A: Let's think step-by-step...
def format_example(example):
    return {
        "text": f"Q: {example['question']}\nA: Let's think step-by-step. {example['answer']}"
    }

formatted_dataset = dataset.map(format_example)
train_data = formatted_dataset["train"]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [2]:
# ✅ STEP 3: Load GPT-2 model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# ✅ STEP 4: Tokenize the dataset
def tokenize(batch):
    tokenized = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in ids]
        for ids in tokenized["input_ids"]
    ]
    return tokenized



tokenized_dataset = train_data.map(tokenize, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [ ]:
# ✅ STEP 5: Define training arguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-gsm8k-cot",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none",
)

# ✅ STEP 6: Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

# # ✅ STEP 7: Save to Google Drive (optional)
# from google.colab import drive
# drive.mount('/content/drive')

# model.save_pretrained("/content/drive/MyDrive/gpt2-gsm8k-cot")
# tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-gsm8k-cot")

# print("✅ Model saved to Google Drive.")


/tmp/ipython-input-8-4214670974.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,2.328400
100,2.026500
150,1.866200
200,1.856500
250,1.818000
300,1.794300
350,1.806700
400,1.799200
450,1.755900
500,1.734500


Step,Training Loss
50,2.328400
100,2.026500
150,1.866200
200,1.856500
250,1.818000
300,1.794300
350,1.806700
400,1.799200
450,1.755900
500,1.734500


In [ ]:
prompt = "Q: If Alice has 3 apples and buys 2 more, how many apples does she have?\nA: Let's think step-by-step."
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))